In [2]:
   %pip install numpy==1.24.3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

# device 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DOG_SKELETON = [
    [0, 1],  # 코에서 이마
    [0, 2],  # 코에서 입꼬리
    [2, 3],  # 입꼬리에서 아랫입술
    [1, 4],  # 이마에서 목
    [4, 5],  # 목에서 오른쪽 앞다리 시작점
    [4, 6],  # 목에서 왼쪽 앞다리 시작점
    [5, 7],  # 오른쪽 앞다리 시작점에서 오른쪽 앞발목
    [6, 8],  # 왼쪽 앞다리 시작점에서 왼쪽 앞발목
    [9, 11],  # 오른쪽 허벅지에서 오른쪽 뒷발목
    [10, 12],  # 왼쪽 허벅지에서 왼쪽 뒷발목
    [4, 13],  # 목에서 꼬리 시작점
    [13, 14],  # 꼬리 시작점에서 꼬리 끝
    [9, 13],  # 오른쪽 허벅지에서 꼬리 시작점 (오른쪽 하체)
    [10, 13], # 왼쪽 허벅지에서 꼬리 시작점 (왼쪽 하체)
    [5, 9],   # 오른쪽 앞다리 시작점에서 오른쪽 허벅지 (오른쪽 몸통)
    [6, 10],  # 왼쪽 앞다리 시작점에서 왼쪽 허벅지 (왼쪽 몸통)
    [5, 6],   # 오른쪽 앞다리 시작점에서 왼쪽 앞다리 시작점 (가슴 윤곽)
    [9, 10]   # 오른쪽 허벅지에서 왼쪽 허벅지 (엉덩이 윤곽)
]


def extract_keypoints(keypoints):
    extracted = []
    for i in range(1, 16):  # 1부터 15까지의 키포인트
        point = keypoints.get(str(i))
        if point is not None:
            extracted.extend([point['x'], point['y']])
        else:
            extracted.extend([0, 0])  # 없는 키포인트는 (0, 0)으로 처리
    return extracted

def extract_skeleton(keypoints):
    skeleton = []
    for start, end in DOG_SKELETON:
        start_point = keypoints.get(str(start+1))
        end_point = keypoints.get(str(end+1))
        if start_point and end_point:
            skeleton.extend([start_point['x'], start_point['y'], end_point['x'], end_point['y']])
        else:
            skeleton.extend([0, 0, 0, 0])
    return skeleton

def load_json_files(base_folder):
    keypoints_data = []
    skeleton_data = []
    labels = []
    metadata = []
    total_files = 0
    processed_files = 0
    skipped_files = 0
    class_names = []

    for action_folder in os.listdir(base_folder):
        action_path = os.path.join(base_folder, action_folder)
        if os.path.isdir(action_path):
            class_name = action_folder.upper()
            if class_name not in class_names:
                class_names.append(class_name)
            class_index = class_names.index(class_name)

            for filename in os.listdir(action_path):
                if filename.endswith('.json'):
                    total_files += 1
                    file_path = os.path.join(action_path, filename)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        try:
                            json_data = json.load(f)
                            keypoints_sequence = []
                            skeleton_sequence = []
                            for annotation in json_data['annotations']:
                                keypoints = extract_keypoints(annotation['keypoints'])
                                skeleton = extract_skeleton(annotation['keypoints'])
                                if keypoints and skeleton:
                                    keypoints_sequence.append(keypoints)
                                    skeleton_sequence.append(skeleton)
                            if keypoints_sequence and skeleton_sequence:
                                keypoints_data.append(keypoints_sequence)
                                skeleton_data.append(skeleton_sequence)
                                labels.append(class_index)
                                metadata.append({
                                    'pain': json_data['metadata']['owner']['pain'],
                                    'disease': json_data['metadata']['owner']['disease'],
                                    'emotion': json_data['metadata']['owner']['emotion'],
                                    'abnormal_action': json_data['metadata']['inspect']['abnormalAction']
                                })
                                processed_files += 1
                            else:
                                print(f"경고: {filename}에서 유효한 시퀀스를 추출하지 못했습니다.")
                                skipped_files += 1
                        except Exception as e:
                            print(f"오류 발생: {filename} 처리 중 {str(e)}")
                            skipped_files += 1

    print(f"총 파일 수: {total_files}")
    print(f"처리된 파일 수: {processed_files}")
    print(f"건너뛴 파일 수: {skipped_files}")
    print(f"클래스 목록: {class_names}")

    if not keypoints_data:
        raise ValueError("로드된 데이터가 없습니다. 데이터 경로와 파일을 확인하세요.")

    return keypoints_data, skeleton_data, labels, metadata, class_names

def pad_sequences(sequences, max_length):
    padded_sequences = []
    for seq in sequences:
        if len(seq) > max_length:
            padded_sequences.append(seq[:max_length])
        else:
            padding = [seq[-1]] * (max_length - len(seq))
            padded_sequences.append(seq + padding)
    return np.array(padded_sequences)

class ImprovedLSTMModel(nn.Module):
    def __init__(self, keypoint_size, skeleton_size, hidden_size, num_layers, num_classes):
        super(ImprovedLSTMModel, self).__init__()
        self.keypoint_lstm = nn.LSTM(keypoint_size, hidden_size, num_layers, batch_first=True, dropout=0.5)
        self.skeleton_lstm = nn.LSTM(skeleton_size, hidden_size, num_layers, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, keypoints, skeleton):
        _, (h_n_keypoints, _) = self.keypoint_lstm(keypoints)
        _, (h_n_skeleton, _) = self.skeleton_lstm(skeleton)

        combined = torch.cat((h_n_keypoints[-1], h_n_skeleton[-1]), dim=1)
        out = self.dropout(combined)
        out = self.fc(out)
        return out

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

def save_model(epoch, model, optimizer, scheduler, train_loss, val_loss, filename, all_meta, all_class_names):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'keypoint_size': keypoint_size,
        'skeleton_size': skeleton_size,
        'hidden_size': hidden_size,
        'num_layers': num_layers,
        'num_classes': num_classes,
        'all_class_names': all_class_names,
        'metadata': all_meta  # 메타데이터 추가
    }, filename)
    print(f"모델이 '{filename}'로 저장되었습니다.")

# 데이터 로딩
base_folder_train = 'E:\반려동물 구분을 위한 동물 영상\Training'
base_folder_val = 'E:\반려동물 구분을 위한 동물 영상\Validation'

try:
    print("학습 데이터 로딩 중...")
    X_train_keypoints, X_train_skeleton, y_train, meta_train, class_names_train = load_json_files(base_folder_train)
    print("\n검증 데이터 로딩 중...")
    X_val_keypoints, X_val_skeleton, y_val, meta_val, class_names_val = load_json_files(base_folder_val)

    # 학습 데이터와 검증 데이터의 클래스 목록 통합
    all_class_names = list(set(class_names_train + class_names_val))
    num_classes = len(all_class_names)

    print(f"\n로드된 학습 데이터 수: {len(X_train_keypoints)}")
    print(f"로드된 검증 데이터 수: {len(X_val_keypoints)}")
    print(f"총 클래스 수: {num_classes}")
    print(f"클래스 목록: {all_class_names}")
except ValueError as e:
    print(f"오류: {e}")
    exit(1)

# 시퀀스 패딩
max_length = max(max(len(seq) for seq in X_train_keypoints), max(len(seq) for seq in X_val_keypoints))
X_train_keypoints = pad_sequences(X_train_keypoints, max_length)
X_train_skeleton = pad_sequences(X_train_skeleton, max_length)
X_val_keypoints = pad_sequences(X_val_keypoints, max_length)
X_val_skeleton = pad_sequences(X_val_skeleton, max_length)

# 텐서로 변환
X_train_keypoints = torch.FloatTensor(X_train_keypoints)
X_train_skeleton = torch.FloatTensor(X_train_skeleton)
y_train = torch.LongTensor(y_train)
X_val_keypoints = torch.FloatTensor(X_val_keypoints)
X_val_skeleton = torch.FloatTensor(X_val_skeleton)
y_val = torch.LongTensor(y_val)

# 데이터로더 생성
train_dataset = TensorDataset(X_train_keypoints, X_train_skeleton, y_train)
val_dataset = TensorDataset(X_val_keypoints, X_val_skeleton, y_val)
full_dataset = ConcatDataset([train_dataset, val_dataset])

# IndexedDataset 클래스 정의
class IndexedDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        data = self.dataset[index]
        return (index,) + data

    def __len__(self):
        return len(self.dataset)

# 기존 데이터셋을 IndexedDataset으로 감싸기
indexed_train_dataset = IndexedDataset(train_dataset)
indexed_val_dataset = IndexedDataset(val_dataset)
indexed_full_dataset = IndexedDataset(full_dataset)

# 새로운 DataLoader 생성
train_loader = DataLoader(indexed_train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(indexed_val_dataset, batch_size=32, shuffle=False)
full_loader = DataLoader(indexed_full_dataset, batch_size=32, shuffle=False)

# 모델 파라미터 (이전과 동일)
keypoint_size = X_train_keypoints.shape[2]
skeleton_size = X_train_skeleton.shape[2]
hidden_size = 128
num_layers = 2
num_classes = len(all_class_names)

# 저장된 모델 확인 및 로드
latest_checkpoint = max([f for f in os.listdir('.') if f.startswith('improved_lstm_model_dog_epoch_')], default=None)
start_epoch = 0

if latest_checkpoint:
    print(f"최신 체크포인트 발견: {latest_checkpoint}")
    checkpoint = torch.load(latest_checkpoint, map_location=device)
    start_epoch = checkpoint['epoch'] + 1

    keypoint_size = checkpoint['keypoint_size']
    skeleton_size = checkpoint['skeleton_size']
    hidden_size = checkpoint['hidden_size']
    num_layers = checkpoint['num_layers']
    num_classes = checkpoint['num_classes']
    all_class_names = checkpoint['all_class_names']
    all_meta = checkpoint.get('metadata', meta_train + meta_val)  # 메타데이터 로드

    model = ImprovedLSTMModel(keypoint_size, skeleton_size, hidden_size, num_layers, num_classes).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    print(f"체크포인트에서 학습 재개: 에포크 {start_epoch}")
else:
    print("새로운 학습 시작")
    model = ImprovedLSTMModel(keypoint_size, skeleton_size, hidden_size, num_layers, num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)
    all_meta = meta_train + meta_val

criterion = nn.CrossEntropyLoss()

# 학습
num_epochs = 100
patience = 20
early_stopping = EarlyStopping(patience=patience, verbose=True, path='best_model.pt')

for epoch in tqdm(range(start_epoch, num_epochs), desc="Epochs"):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for indices, batch_keypoints, batch_skeleton, batch_y in train_loader:
        batch_keypoints, batch_skeleton, batch_y = batch_keypoints.to(device), batch_skeleton.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_keypoints, batch_skeleton)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    train_loss = total_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    # 검증
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for indices, batch_keypoints, batch_skeleton, batch_y in val_loader:
            batch_keypoints, batch_skeleton, batch_y = batch_keypoints.to(device), batch_skeleton.to(device), batch_y.to(device)
            outputs = model(batch_keypoints, batch_skeleton)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += batch_y.size(0)
            val_correct += (predicted == batch_y).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * val_correct / val_total

    print(f'에포크 [{epoch+1}/{num_epochs}], 학습 손실: {train_loss:.4f}, 학습 정확도: {train_accuracy:.2f}%, 검증 손실: {val_loss:.4f}, 검증 정확도: {val_accuracy:.2f}%')

    # 10 에포크마다 모델 저장
    if (epoch + 1) % 10 == 0:
        save_model(epoch, model, optimizer, scheduler, train_loss, val_loss, f'improved_lstm_model_dog_epoch_{epoch+1}.pt', all_meta, all_class_names)

    # Early Stopping 체크
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    scheduler.step(val_loss)

print("학습 완료")

# 최종 모델 저장
save_model(epoch, model, optimizer, scheduler, train_loss, val_loss, 'improved_lstm_model_dog_final.pt', all_meta, all_class_names)

# 최상의 모델 로드
best_model = torch.load('best_model.pt', map_location=device)
model.load_state_dict(best_model['model_state_dict'])
all_meta = best_model.get('metadata', all_meta)
all_class_names = best_model.get('all_class_names', all_class_names)

# 전체 데이터셋에 대한 평가
full_dataset = ConcatDataset([train_dataset, val_dataset])
full_loader = DataLoader(full_dataset, batch_size=32, shuffle=False)

model.eval()
correct = 0
total = 0
all_predictions = []
all_true_labels = []
all_indices = []


# 전체 데이터셋 평가
with torch.no_grad():
    for indices, batch_keypoints, batch_skeleton, batch_y in full_loader:
        batch_keypoints, batch_skeleton, batch_y = batch_keypoints.to(device), batch_skeleton.to(device), batch_y.to(device)
        outputs = model(batch_keypoints, batch_skeleton)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
        all_predictions.extend(predicted.cpu().numpy())
        all_true_labels.extend(batch_y.cpu().numpy())
        all_indices.extend(indices.numpy())

print(f'전체 데이터셋 정확도: {100 * correct / total:.2f}%')

# 예측 결과와 메타데이터 함께 표시
for i, pred, true in zip(all_indices, all_predictions, all_true_labels):
    pred_class = all_class_names[pred]
    true_class = all_class_names[true]
    metadata = all_meta[i]
    print(f"샘플 {i+1}:")
    print(f"  예측: {pred_class}, 실제: {true_class}")
    print(f"  메타데이터:")
    print(f"    통증: {metadata['pain']}")
    print(f"    질병: {metadata['disease']}")
    print(f"    감정: {metadata['emotion']}")
    print(f"    비정상 행동: {metadata['abnormal_action']}")
    print()

print("평가 완료")

학습 데이터 로딩 중...
총 파일 수: 39537
처리된 파일 수: 39537
건너뛴 파일 수: 0
클래스 목록: ['BODYLOWER', 'BODYSCRATCH', 'BODYSHAKE', 'FEETUP', 'FOOTUP', 'HEADING', 'LYING', 'MOUNTING', 'SIT', 'TAILING', 'TAILLOW', 'TURN', 'WALKRUN']

검증 데이터 로딩 중...
총 파일 수: 4949
처리된 파일 수: 4949
건너뛴 파일 수: 0
클래스 목록: ['BODYLOWER', 'BODYSCRATCH', 'BODYSHAKE', 'FEETUP', 'FOOTUP', 'HEADING', 'LYING', 'MOUNTING', 'SIT', 'TAILING', 'TAILLOW', 'TURN', 'WALKRUN']

로드된 학습 데이터 수: 39537
로드된 검증 데이터 수: 4949
총 클래스 수: 13
클래스 목록: ['MOUNTING', 'FEETUP', 'BODYLOWER', 'TURN', 'SIT', 'BODYSHAKE', 'TAILING', 'HEADING', 'BODYSCRATCH', 'TAILLOW', 'LYING', 'WALKRUN', 'FOOTUP']
최신 체크포인트 발견: improved_lstm_model_dog_epoch_90.pt


C:\Users\wjsgh\AppData\Local\Temp\ipykernel_24176\1659742754.py:273: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(latest_checkpoint, map_location=de

체크포인트에서 학습 재개: 에포크 90


Epochs:  10%|█         | 1/10 [03:06<27:54, 186.07s/it]

에포크 [91/100], 학습 손실: 1.6033, 학습 정확도: 49.13%, 검증 손실: 1.7102, 검증 정확도: 44.55%
Validation loss decreased (inf --> 1.710159). Saving model ...


...

에포크 [92/100], 학습 손실: 1.6014, 학습 정확도: 48.86%, 검증 손실: 1.7084, 검증 정확도: 44.37%
Validation loss decreased (1.710159 --> 1.708443). Saving model ...


Epochs:  30%|███       | 3/10 [08:59<20:49, 178.45s/it]

에포크 [93/100], 학습 손실: 1.5951, 학습 정확도: 49.03%, 검증 손실: 1.7031, 검증 정확도: 44.66%
Validation loss decreased (1.708443 --> 1.703091). Saving model ...


Epochs:  30%|███       | 3/10 [09:59<23:17, 199.69s/it]


KeyboardInterrupt: 